Задание теме «Коллаборативная фильтрация» / ПАКЕТ SURPRISE

- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- RMSE посчитать на основе CrossValidation (5 фолдов)

получите RMSE на тестовом сете 0.87 и ниже

In [1]:
import pandas as pd
import numpy as np
import re

from warnings import filterwarnings 
filterwarnings('ignore')

In [2]:
from surprise import KNNWithMeans
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate

Movies

In [91]:
movies = pd.read_csv('movies.csv')
movies.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


Представим себе, что каждый жанр, указанный в поле genres -  это тэг,  и сделаем:

In [92]:
genres_vect = movies["genres"].str.get_dummies("|").iloc[:,1:]

In [93]:
genres_vect.head(1)

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [94]:
i             = 0
col_s         = ['movieId','tag']
genres_as_tag = pd.DataFrame(columns =col_s)


for i in range (19):
    df                      = pd.DataFrame(list(zip(movies.movieId, genres_vect.iloc[:,i])))
    df.columns              = col_s
    df.loc[df.tag>0, 'tag'] = genres_vect.columns[i]
    genres_as_tag           = pd.concat([genres_as_tag, df],axis = 0)

genres_as_tag = genres_as_tag.loc[genres_as_tag.tag != 0]

In [95]:
genres_as_tag.head(1)

,movieId,tag
5,6,Action


сформируем дополнительные признаки - год выпуска фильма и период выпуска фильма (с точностью до 5-10 лет)

In [97]:
def set_year(row):
    z = re.findall('(\d\d\d\d)', row.title)
    if len(z) == 0 or z == 0 or z == None:
        z = 9999
    else:
        z = int(z[-1])
    return z    

movies['year_of_film']= movies.apply(set_year, axis=1)

In [98]:
movies['epoch_of_film'] = pd.cut(movies.year_of_film, [1900, 1960, 1970, 1980, 1990, 2000,
                                                       2005, 2010, 2015, 10000]) 
movies['epoch_of_film'].value_counts()

(2010, 2015]     10713
(2005, 2010]      8176
(1900, 1960]      8040
(1990, 2000]      7062
(2000, 2005]      5417
(2015, 10000]     5295
(1980, 1990]      4933
(1970, 1980]      4738
(1960, 1970]      3605
Name: epoch_of_film, dtype: int64

Эпоха фильма тоже может быть тэгом, почему бы и нет:

In [105]:
epoch_as_tag = movies.iloc[:,[0,4]]
epoch_as_tag.columns  = col_s
epoch_as_tag.head()

,movieId,tag
0,1,"(1990, 2000]"
1,2,"(1990, 2000]"
2,3,"(1990, 2000]"
3,4,"(1990, 2000]"
4,5,"(1990, 2000]"


Соединим все синтетические тэги и оставим до лучших временЖ

In [106]:
movie_tag = pd.concat([genres_as_tag, epoch_as_tag],axis = 0)
movie_tag.head()

,movieId,tag
5,6,Action
8,9,Action
9,10,Action
14,15,Action
19,20,Action


In [107]:
movie_tag.shape

(159939, 2)

tags

In [108]:
tags = pd.read_csv('tags.csv')

In [109]:
tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


In [110]:
tags.tag.value_counts().head()

sci-fi         9400
atmospheric    6430
action         6219
comedy         5923
surreal        5299
Name: tag, dtype: int64

исключаем пустые записи

In [111]:
tags = tags[~tags.tag.isna()]

приводим все теги к нижнему регистру

In [112]:
tags['tag'] = tags['tag'].str.lower()

если наименование жанра содержится в теге - оставляем типовое поле о принадлежности тега к жанру, жанр уже отражен в базе movies тэгов

In [113]:
for i in genres_vect.columns.str.lower().tolist():
    tags.loc[tags.tag ==i, 'tag'] = 'genres_tag'

In [114]:
tags.tag.value_counts().head()

genres_tag         60851
atmospheric         6995
surreal             5572
funny               5560
based on a book     5414
Name: tag, dtype: int64

если время выпуска фильма содержится в теге - оставляем типовое поле о принадлежности тега ко времение, "эпоха" уже отражена в базе movies

In [115]:
tags[tags.tag.str.contains('0s')].head()

,userId,movieId,tag,timestamp
292,206,541,80s,1527550262
293,206,2115,80s,1527550065
295,206,2402,80s,1527652380
296,206,2403,80s,1527652469
297,206,3740,80s,1527550214


In [116]:
tags.loc[tags.tag.str.contains('0s'),  'tag'] = 'epoch_tag'
tags.loc[tags.tag.str.contains("0's"), 'tag'] = 'epoch_tag'

В списке тэгов очень часто встречаются имена и фамилии. Это либо имена актеров и создателей фильма или имена главных героев фильма. Введем универсальный признак star_tag, обозначающий, что в тэге есть имя Звезды

In [117]:
men_names    = pd.read_html('https://audio-class.ru/names/mens-names.php', 
                         encoding = 'utf-8')[0].iloc[:,1].str.lower().tolist()
women_names  = pd.read_html('https://audio-class.ru/names/womens-names.php/', 
                        encoding = 'utf-8')[0].iloc[:,1].str.lower().tolist()
family_names = pd.read_html('https://audio-class.ru/names/last-names-sortable.php', 
                        encoding = 'utf-8')[0].iloc[:,1].str.lower().tolist()

names = men_names + women_names + family_names

In [118]:
n = []
for i in names:
    n += (i.split('/'))
names = n

In [119]:
for i in names:
    tags.loc[tags.tag.str.contains(i), 'tag'] = 'star_tag'

In [120]:
tags[tags.tag=='star_tag'].shape

(365807, 4)

In [121]:
tags.shape, tags.tag.unique().shape

((1108981, 4), (38174,))

Разберемся со словосочетаниями в тэгах, которые встречаются не более 20 раз, 

логика преобразований:
- составляем "мешок" слов из всех словосочетаний "редких тегов" (содержащихся не более 20 раз)
- выбираем Топ 100 самых часто встречающихся слов "из мешка",
- убираем из Топ 100 очевидные предлоги и слова без смысловой нагрузки,
- если в "редком теге" встречается слово из Топ100, заменяем "редкий тэг" на ключевое слово редкого тэга

In [122]:
rare_tags = tags.tag.value_counts()[tags.tag.value_counts() < 20]
rare_tags

promiscuity         19
victim of murder    19
chris marker        19
cafe                19
family reunion      19
                    ..
curgir               1
pygmy                1
irreverent humor     1
any time watch       1
boorish              1
Name: tag, Length: 34552, dtype: int64

In [123]:
rti = tuple(rare_tags.index)

In [124]:
l =[]
for i in rti:
    l.append(re.findall('[\w]+', i))

In [125]:
from collections import Counter

In [126]:
l1 = []
for i in Counter(sum(l, [])).most_common(100):
    l1.append(i[0])
l1[:10]

['the', 'of', 'a', 'movie', 'to', 'in', 's', 'and', 'writer', 'not']

In [127]:
words_bag=('movie', 'writer', 'bad', 'not', 'film', 'great', 'man', 'no', 'director', 'story', 'music', 'ending',
          'plot', 'child', 'book', 'family', 'character', 'woman', 'setting', 'too', 'death', 'sex', 'see', 'funny',
          'afi', 'time', 'based', 'characters', 'better', 'girl', 'up', 'scene', 'relationship', 'animal', 'american',
          'classic', 'one', 'all', 'world', 'self', 'new', 'car', 'protagonist', 'robert', 'you', 'evil', 'than',
          'watch', 'women', 'but', 'soundtrack', 'old', 'de', 'stupid', 'humor', 'human', 'out', 'poor',  'very',
          '13', 'father', 'acting', 'real', 'sexual', 'fight', 'original', 'history', 'boy', 'mother', 'dead', '06', 
          'art', 'hero', 'first', 'kids', 'drug')

In [128]:
for i in range(tags.shape[0]): 
    if tags.iloc[i,2] in rti:
        for w in words_bag:
            if w in tags.iloc[i,2]:
                tags.iloc[i,2] = w

In [129]:
tags.tag.value_counts().head()

star_tag       365807
genres_tag      60851
atmospheric      6995
de               5903
funny            5860
Name: tag, dtype: int64

In [130]:
tags.loc[~tags.tag.isin(tags.tag.value_counts()[tags.tag.value_counts() <= 20].index), 'tag'].value_counts()

star_tag                    365807
genres_tag                   60851
atmospheric                   6995
de                            5903
funny                         5860
                             ...  
unhappy marriage                21
conscience                      21
walked out/didn't finish        21
getaway                         21
jan svankmajer                  21
Name: tag, Length: 3540, dtype: int64

In [131]:
selected_tags = tags[~tags.tag.isin(tags.tag.value_counts()[tags.tag.value_counts() <= 20].index)]

In [132]:
selected_tags = selected_tags.iloc[:,:3]

In [133]:
selected_tags.head(1)

,userId,movieId,tag
0,14,110,epic


ну и объединяем тэги со всеми данными, полученными из файла movies

In [135]:
sum_tags = pd.concat([movie_tag, selected_tags.iloc[:,[1,2]]], axis = 0)
sum_tags.head(1)

,movieId,tag
5,6,Action


ratings

In [136]:
ratings = pd.read_csv('ratings.csv')
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221


In [140]:
ratings_avg = ratings.groupby(['movieId']).rating.median()
ratings_avg.head(1)

movieId
1    4.0
Name: rating, dtype: float64

и выходим на аккуратную базу, в которой есть только сведения для предсказания медианного рейтинга

In [143]:
base_surprise = sum_tags.merge(ratings_avg, 
                               how = 'left', 
                               left_on = ['movieId'], 
                               right_on= ratings_avg.index)
base_surprise.head(1)

,movieId,tag,rating
0,6,Action,4.0


In [144]:
base_surprise = base_surprise[~base_surprise.rating.isna()]

In [145]:
base_surprise.shape

(1179147, 3)

In [146]:
base_surprise.rating.min(), base_surprise.rating.max()

(0.5, 5.0)

Surprise

In [147]:
dataset = pd.DataFrame({
    'uid':    base_surprise.movieId,
    'iid':    base_surprise.tag,
    'rating': base_surprise.rating
})

In [148]:
reader      = Reader(line_format='user item rating', rating_scale=(0.5,5.0))
df_prepared = Dataset.load_from_df(dataset, reader)

SVD

In [150]:
algo_svd = SVD(n_factors=20, n_epochs=20)

In [151]:
cv_svd = cross_validate(algo_svd, df_prepared, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.2994  0.3008  0.2822  0.2892  0.3072  0.2957  0.0089  
Fit time          24.02   24.57   24.54   24.95   24.77   24.57   0.31    
Test time         1.98    1.96    1.90    1.99    1.71    1.91    0.10    


SlopeOne

In [153]:
from surprise import SlopeOne

In [154]:
algo_SlopeOne = SlopeOne()
cv_SlopeOne   = cross_validate(algo_SlopeOne, df_prepared, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0734  0.0750  0.0724  0.0735  0.0789  0.0747  0.0023  
Fit time          18.91   19.86   19.63   19.64   19.27   19.46   0.33    
Test time         74.87   75.81   78.25   74.89   76.12   75.99   1.23    
